In [1]:
using CSV, DataFrames, Plots, Statistics

In [2]:
cps = DataFrame(CSV.File("c:\\data\\Bounds\\cps.csv")) # <-change this to the right directory

## Cleaning zero wage

cps = cps[cps.wage .> 0, :]
first(cps,5)

In [3]:
# Numerical value for education

#1.Creating a dictionary

educ_dict = Dict( 
    "grade 11" => 11, 
    "some college but no degree" => 13, 
    "associate's degree, academic program" => 14,
    "grade 10" => 10, 
    "grades 7 or 8" => 8, 
    "grades 1, 2, 3, or 4" => 4, 
    "associate's degree, occupational/vocational program" => 14, 
    "high school diploma or equivalent" => 12, 
    "grade 9" => 9, 
    "none or preschool" => 0, 
    "doctorate degree" => 21,
    "bachelor's degree" => 16, 
    "master's degree" => 14, 
    "grades 5 or 6" => 6, 
    "professional school degree" => 14, 
    "12th grade, no diploma" => 12
    );

In [4]:
#2. transforming cps.educ
f(x) = educ_dict[x]
cps.educ_num = f.(cps.educ)

#3. log of wage
cps.log_wage = log.(cps.wage)
first(cps,3)

In [5]:
Nobs, c = size(cps)
print("The original data contains ", Nobs, " observations and ",c, " columns.")

In [6]:
wage_quantiles = quantile(cps[!,:wage],[0.0, 0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0], sorted=false);
#wage_quantiles[1] +=-1
wage_quantiles[end] +=1

In [7]:
lower(x) = wage_quantiles[sum(wage_quantiles .<=x)]
upper(x) = wage_quantiles[sum(wage_quantiles .<=x)+1]

cps.wage_lower = lower.(cps.wage)
cps.wage_upper = upper.(cps.wage)

cps.logwage_lower = log.(cps.wage_lower)
cps.logwage_upper = log.(cps.wage_upper)

first(cps,3)

In [8]:
df = cps[1:20,:]

In [9]:
describe(df)

In [10]:
include("C:\\Users\\arie\\OneDrive - University of Pittsburgh\\Documents\\Bounds.jl\\Bounds.jl\\src\\setBLP.jl")

In [11]:
using Main.setBLP

In [12]:
x = df[:,:age];
yl = df[:,:logwage_lower];
yu = df[:,:logwage_upper];
xyl = x.*yl;
xyu = x.*yu;

In [13]:
vl = Vertex([yl[1],xyl[1]])
vu = Vertex([yu[1],xyu[1]])
P = Polygon([vl,vu])
sortPolygon!(P)

for j = 2:20
    vl = Vertex([yl[j],xyl[j]])
    vu = Vertex([yu[j],xyu[j]])
    P0 = Polygon([vl,vu])
    sortPolygon!(P0)
    P +=P0
    sortPolygon!(P)
end

In [14]:
scatterPolygon(P)

In [15]:
plotPolygon(P)

In [16]:
P